In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/home/ec2-user/Face-Classification-Project/Facial_Impression_Recognition_Calassification'

In [3]:
from cnnClassifier.utils import load_object
import keras

2023-08-03 21:27:53.021442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 21:27:53.954243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    path_of_model_json: Path   
    test_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [6]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])  

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= self.config.training.trained_model_path,
            path_of_model_json=  self.config.training.trained_model_json_path,    
            test_data= Path('artifacts/data_transformation/test'),
            all_params = self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
            )
        
        return eval_config

In [7]:
import sys
from cnnClassifier.exception import CustomException
from cnnClassifier.utils import save_json, load_model


In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _test_generator(self):

        datagenerator_kwargs = dict(
            # Dividing the pixels by 255 for normalization  => range(0,1)
            # Scaling the pixels value in range(-1,1) by subtracting 0.5 and multiply 2
            rescale= ((1./255) - 0.5)*2           
            
            
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],            
            batch_size = self.config.params_batch_size,
            interpolation= 'bilinear',
            color_mode="grayscale"
        )

        datagenerator = keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
        )
        
        ## NOTE: subset is added when we use validation_split, where directory for both training and validation is the same.
        ## NOTE: shuffle= False for test as we want to check the performance model using predict_generator 
        self.test_generator = datagenerator.flow_from_directory(
            directory= self.config.test_data,            
            shuffle= False,            
            **dataflow_kwargs
            )
       
         
    def evaluation(self):
        model = load_model(h5_path= self.config.path_of_model, json_path= self.config.path_of_model_json)    
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=self.config.all_params['LEARNING_RATE']),
            loss = keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
                           )    
        self._test_generator()
        self.score = model.evaluate(self.test_generator)

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path='scores.json', data=scores)


In [ ]:
try:
    config = configurationManeger()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise CustomException(e, sys)


2023-08-03 21:27:54.920618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-03 21:27:54.957220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-03 21:27:54.957503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3589 images belonging to 7 classes.


2023-08-03 21:27:56.903754: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:437] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-08-03 21:27:56.903834: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:441] Memory usage: 4325376 bytes free, 16935419904 bytes total.
2023-08-03 21:27:56.903868: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at conv_ops_impl.h:770 : UNIMPLEMENTED: DNN library is not found.
